# Crypto Data
## Setup

In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from data_processing import download_data

## Price Data

In [154]:
tickers = [coin + "-USD" for coin in ["BTC", "ETH", "XRP", "BNB", "SOL", "TRX", "DOGE", "BCH", "ADA", "HYPE"]]

start = "2024-02-11"
end = "2026-02-11"

dfs = []
for ticker in tickers:
    df = download_data(ticker, start, end, save=True)
    df.rename({col: col + " " + ticker[:3] for col in df.columns if col != "Date"}, axis=1, inplace=True)
    df.set_index("Date", inplace=True)
    dfs.append(df)

df = pd.concat(dfs, axis=1)
df.sort_index(inplace=True)

/Users/fgias/opt/anaconda3/envs/stgan/lib/python3.12/site-packages/yfinance/scrapers/history.py:172: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/Users/fgias/opt/anaconda3/envs/stgan/lib/python3.12/site-packages/yfinance/scrapers/history.py:172: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/Users/fgias/opt/anaconda3/envs/stgan/lib/python3.12/site-packages/yfinance/scrapers/history.py:172: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now = pd.Timestamp.utcnow()
/Users/fgias/opt/anaconda3/envs/stgan/lib/python3.12/site-packages/yfinance/scrapers/history.py:172: Pandas4Warning: Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.
  dt_now 

In [155]:
df

,Adj Close BTC,Adj Close ETH,Adj Close XRP,Adj Close BNB,Adj Close SOL,Adj Close TRX,Adj Close DOG,Adj Close BCH,Adj Close ADA,Adj Close HYP
Date,,,,,,,,,,
2024-02-11,48293.917969,2507.570557,0.526084,320.787537,107.495285,0.124278,0.081193,276.501923,0.541410,0.000027
2024-02-12,49958.222656,2658.115967,0.531897,327.944611,111.992752,0.125058,0.082345,280.857941,0.560031,0.000027
2024-02-13,49742.441406,2642.185303,0.524835,324.868225,112.579308,0.126942,0.081089,270.852081,0.544996,0.000025
2024-02-14,51826.695312,2777.902344,0.538401,334.245972,116.976326,0.129684,0.085543,281.261658,0.577302,0.000023
2024-02-15,51938.554688,2824.378906,0.562755,354.710815,113.699409,0.131169,0.085356,270.665924,0.608363,0.000023
...,...,...,...,...,...,...,...,...,...,...
2026-02-06,70555.390625,2063.386963,1.471523,657.421509,87.464775,0.273972,0.098489,523.778015,0.276234,NaN
2026-02-07,69281.968750,2090.549316,1.424204,647.350281,87.639832,0.278085,0.098277,524.912842,0.272365,NaN
2026-02-08,70264.726562,2088.762939,1.432185,640.253662,87.044106,0.277825,0.096738,527.000610,0.270263,NaN


In [156]:
# Save
np.save("stgan-algo/STGAN/crypto/data/data.npy", df)

In [157]:
# Load
data_tensor = torch.tensor(np.load("stgan-algo/STGAN/crypto/data/data.npy"), dtype=torch.float)
data_tensor.shape

torch.Size([731, 10])

In [158]:
data_tensor

tensor([[4.8294e+04, 2.5076e+03, 5.2608e-01,  ..., 2.7650e+02, 5.4141e-01,
         2.7000e-05],
        [4.9958e+04, 2.6581e+03, 5.3190e-01,  ..., 2.8086e+02, 5.6003e-01,
         2.7000e-05],
        [4.9742e+04, 2.6422e+03, 5.2483e-01,  ..., 2.7085e+02, 5.4500e-01,
         2.5000e-05],
        ...,
        [7.0265e+04, 2.0888e+03, 1.4322e+00,  ..., 5.2700e+02, 2.7026e-01,
                nan],
        [7.0121e+04, 2.1036e+03, 1.4361e+00,  ..., 5.3280e+02, 2.6971e-01,
                nan],
        [6.8794e+04, 2.0195e+03, 1.3995e+00,  ..., 5.2424e+02, 2.6167e-01,
                nan]])

## Distances: Use Correlations

In [159]:
# Use simple intuition: distance = 1 - correlation
dists =  1 - df.corr()

In [160]:
dists

,Adj Close BTC,Adj Close ETH,Adj Close XRP,Adj Close BNB,Adj Close SOL,Adj Close TRX,Adj Close DOG,Adj Close BCH,Adj Close ADA,Adj Close HYP
Adj Close BTC,0.000000,0.646017,0.068168,0.288464,0.466915,0.097569,0.403199,0.474615,0.338424,0.471079
Adj Close ETH,0.646017,0.000000,0.788019,0.479445,0.382933,0.754425,0.620421,0.354406,0.625543,0.482023
Adj Close XRP,0.068168,0.788019,0.000000,0.362027,0.598060,0.083650,0.450556,0.596749,0.317911,0.713319
Adj Close BNB,0.288464,0.479445,0.362027,0.000000,0.650856,0.235817,0.752889,0.303412,0.787635,0.720001
Adj Close SOL,0.466915,0.382933,0.598060,0.650856,0.000000,0.714462,0.172209,0.655128,0.295593,0.656550
Adj Close TRX,0.097569,0.754425,0.083650,0.235817,0.714462,0.000000,0.603316,0.468965,0.515483,1.525256
Adj Close DOG,0.403199,0.620421,0.450556,0.752889,0.172209,0.603316,0.000000,0.707398,0.136569,0.268004
Adj Close BCH,0.474615,0.354406,0.596749,0.303412,0.655128,0.468965,0.707398,0.000000,0.791242,0.370146
Adj Close ADA,0.338424,0.625543,0.317911,0.787635,0.295593,0.515483,0.136569,0.791242,0.000000,0.384402
Adj Close HYP,0.471079,0.482023,0.713319,0.720001,0.656550,1.525256,0.268004,0.370146,0.384402,0.000000


In [161]:
dists.to_csv("stgan-algo/STGAN/crypto/data/node_dist.txt", sep=" ", header=False, index=False)

In [162]:
dists.reset_index(drop=True, inplace=True)
dists.columns = range(dists.shape[1])
dists

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.646017,0.068168,0.288464,0.466915,0.097569,0.403199,0.474615,0.338424,0.471079
1,0.646017,0.000000,0.788019,0.479445,0.382933,0.754425,0.620421,0.354406,0.625543,0.482023
2,0.068168,0.788019,0.000000,0.362027,0.598060,0.083650,0.450556,0.596749,0.317911,0.713319
3,0.288464,0.479445,0.362027,0.000000,0.650856,0.235817,0.752889,0.303412,0.787635,0.720001
4,0.466915,0.382933,0.598060,0.650856,0.000000,0.714462,0.172209,0.655128,0.295593,0.656550
5,0.097569,0.754425,0.083650,0.235817,0.714462,0.000000,0.603316,0.468965,0.515483,1.525256
6,0.403199,0.620421,0.450556,0.752889,0.172209,0.603316,0.000000,0.707398,0.136569,0.268004
7,0.474615,0.354406,0.596749,0.303412,0.655128,0.468965,0.707398,0.000000,0.791242,0.370146
8,0.338424,0.625543,0.317911,0.787635,0.295593,0.515483,0.136569,0.791242,0.000000,0.384402
9,0.471079,0.482023,0.713319,0.720001,0.656550,1.525256,0.268004,0.370146,0.384402,0.000000


## Adjacency Matrix

In [163]:
k = 5
adj = pd.DataFrame([], columns=range(k))
for i in range(dists.shape[1]):
    adj.loc[i] = list(dists.loc[i].sort_values()[:k].index)
adj

,0,1,2,3,4
0,0,2,5,3,8
1,1,7,4,3,9
2,2,0,5,8,3
3,3,5,0,7,2
4,4,6,8,1,0
5,5,2,0,3,7
6,6,8,4,9,0
7,7,3,1,9,5
8,8,6,4,2,0
9,9,6,7,8,0


In [164]:
adj.to_csv("stgan-algo/STGAN/crypto/data/node_adjacent.txt", sep=" ", header=False, index=False)

## Node Subgraph

In [165]:
stdev = np.array(dists).std()
stdev

0.28525380343886514

In [166]:
W = np.exp(-dists**2/stdev**2)
for i in range(len(W)):
    W.loc[i,i] = 0   
W

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.005923,0.944492,0.359646,0.068615,8.895911e-01,0.135618,0.062766,0.244746,6.539866e-02
1,0.005923,0.000000,0.000485,0.059310,0.164950,9.167308e-04,0.008822,0.213608,0.008156,5.753033e-02
2,0.944492,0.000485,0.000000,0.199745,0.012330,9.175991e-01,0.082514,0.012570,0.288783,1.924219e-03
3,0.359646,0.059310,0.199745,0.000000,0.005483,5.048865e-01,0.000943,0.322593,0.000489,1.710551e-03
4,0.068615,0.164950,0.012330,0.005483,0.000000,1.886008e-03,0.694572,0.005120,0.341705,5.003951e-03
5,0.889591,0.000917,0.917599,0.504887,0.001886,0.000000e+00,0.011410,0.067016,0.038173,3.830756e-13
6,0.135618,0.008822,0.082514,0.000943,0.694572,1.140970e-02,0.000000,0.002134,0.795158,4.136581e-01
7,0.062766,0.213608,0.012570,0.322593,0.005120,6.701581e-02,0.002134,0.000000,0.000456,1.856723e-01
8,0.244746,0.008156,0.288783,0.000489,0.341705,3.817338e-02,0.795158,0.000456,0.000000,1.626802e-01
9,0.065399,0.057530,0.001924,0.001711,0.005004,3.830756e-13,0.413658,0.185672,0.162680,0.000000e+00


In [167]:
node_subgraph = torch.empty((len(tickers), k, k))
node_subgraph.shape

torch.Size([10, 5, 5])

In [168]:
for i in range(len(tickers)):
    node_subgraph[i,:,:] = torch.from_numpy(W.loc[adj.loc[i], adj.loc[i]].to_numpy())
node_subgraph.shape

torch.Size([10, 5, 5])

In [169]:
node_subgraph[0,:,:]

tensor([[0.0000e+00, 9.4449e-01, 8.8959e-01, 3.5965e-01, 2.4475e-01],
        [9.4449e-01, 0.0000e+00, 9.1760e-01, 1.9975e-01, 2.8878e-01],
        [8.8959e-01, 9.1760e-01, 0.0000e+00, 5.0489e-01, 3.8173e-02],
        [3.5965e-01, 1.9975e-01, 5.0489e-01, 0.0000e+00, 4.8855e-04],
        [2.4475e-01, 2.8878e-01, 3.8173e-02, 4.8855e-04, 0.0000e+00]])

In [170]:
np.save("stgan-algo/STGAN/crypto/data/node_subgraph.npy", node_subgraph.numpy())

## Extra Features

In [171]:
time_features = np.zeros([data_tensor.shape[0], 7])
time_features = pd.DataFrame(time_features, dtype=int)

In [172]:
# Day one-hot encoding
week_length = 7
starting_day = 6 # The starting day, Feb 11, 2024, is Sunday
for i in range(len(time_features)):
    weekday = divmod(i, week_length)[1]
    weekday = divmod(weekday + starting_day, week_length)[1]
    time_features.loc[i, weekday] = 1

In [173]:
time_features

,0,1,2,3,4,5,6
0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
726,0,0,0,0,1,0,0
727,0,0,0,0,0,1,0
728,0,0,0,0,0,0,1
729,1,0,0,0,0,0,0


In [174]:
time_features.to_csv("stgan-algo/STGAN/crypto/data/time_features.txt", sep=" ", header=False, index=False)